# Downscaling Multi-Model Climate Projection Ensembles with Deep Learning (DeepESD): Contribution to CORDEX EUR-44

***Geoscientific Model Development***

**J. Baño-Medina, R. Manzanas, E. Cimadevilla, J. Fernández, J. González-Abad, A.S. Cofiño, and J.M. Gutiérrez**

This notebook reproduces the results presented in **Downscaling Multi-Model Climate Projection Ensembles with Deep Learning (DeepESD): Contribution to CORDEX EUR-44**, submitted to *Geoscientific Model Development* by *J. Baño-Medina, R. Manzanas, E. Cimadevilla, J. Fernández, J. González-Abad, A.S. Cofiño and J.M. Gutiérrez*. 
This paper presents *DeepESD*, the first dataset of high-resolution (0.5º) climate change projections (up to 2100) of daily precipitation and temperature over Europe obtained with deep learning techniques (in particular convolutional neural networks) from an ensemble of eight global climate models from the Coupled Model Intercomparison Project version 5 (CMIP5).

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparing-the-R-environment-and-working-directories" data-toc-modified-id="Preparing-the-R-environment-and-working-directories-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparing the R environment and working directories</a></span><ul class="toc-item"><li><span><a href="#Convolutional-Neural-Networks-(CNNs)" data-toc-modified-id="Convolutional-Neural-Networks-(CNNs)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Convolutional Neural Networks (CNNs)</a></span></li></ul></li><li><span><a href="#DeepESD" data-toc-modified-id="DeepESD-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>DeepESD</a></span><ul class="toc-item"><li><span><a href="#Preparing-the-predictor-datasets" data-toc-modified-id="Preparing-the-predictor-datasets-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Preparing the predictor datasets</a></span></li><li><span><a href="#Precipitation-downscaling" data-toc-modified-id="Precipitation-downscaling-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Precipitation downscaling</a></span></li><li><span><a href="#Temperature-downscaling" data-toc-modified-id="Temperature-downscaling-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Temperature downscaling</a></span></li></ul></li><li><span><a href="#Dynamical-climate-models" data-toc-modified-id="Dynamical-climate-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dynamical climate models</a></span><ul class="toc-item"><li><span><a href="#Ensemble-of-Global-Climate-Models-(GCMs)" data-toc-modified-id="Ensemble-of-Global-Climate-Models-(GCMs)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Ensemble of Global Climate Models (GCMs)</a></span></li><li><span><a href="#Ensemble-of-Regional-Climate-Models-(RCMs)" data-toc-modified-id="Ensemble-of-Regional-Climate-Models-(RCMs)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Ensemble of Regional Climate Models (RCMs)</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#Ensemble-mean-and-bias-with-respect-to-E-OBS" data-toc-modified-id="Ensemble-mean-and-bias-with-respect-to-E-OBS-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Ensemble mean and bias with respect to E-OBS</a></span></li><li><span><a href="#Climate-change-signals" data-toc-modified-id="Climate-change-signals-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Climate change signals</a></span></li><li><span><a href="#Time-series" data-toc-modified-id="Time-series-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Time-series</a></span></li></ul></li><li><span><a href="#Technical-specifications" data-toc-modified-id="Technical-specifications-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Technical specifications</a></span></li></ul></div>

[next notebook](2022_Bano_DeepESD_GMD.ipynb)
<div class="alert alert-block alert-info">
<b>Note:</b> This notebook was run on a machine with the following technical specifications:

- Operating system: Ubuntu 18.04.3 LTS (64 bits)
- Memory: 60 GiB
- Processor: 2x Intel(R) Xeon(R) CPU E5-2670 0 @ 2.60GHz (16 cores, 32 threads)
</div>

## Preparing the R environment and working directories

This notebook is written in the free programming language `R` (version 3.6.1) and builds on [`climate4R`](https://doi.org/10.1016/j.envsoft.2018.09.009) (hereafter C4R), a suite of `R` packages developed by the Santander Meteorology Group for transparent climate data access, post processing (including bias correction and downscaling) and visualization. For details on C4R, the interested reader is referred to [Iturbide et al. 2019](https://doi.org/10.1016/j.envsoft.2018.09.009).

In particular, the following C4R libraries are used along the notebook:


 * `loadeR` (v1.7.0) for data loading,
 * `loadeR.2nc` (v0.1.1) for data loading, 
 * `transformeR` (v2.1.0) for data manipulation, 
 * [`downscaleR`](https://doi.org/10.5194/gmd-13-1711-2020) (v3.3.2) for downscaling and
 * [`downscaleR.keras`](https://doi.org/10.5194/gmd-13-2109-2020) (v1.0.0) for downscaling with neural networks and
 * `visualizeR` (v1.6.0) data visualization
 * `climate4R.UDG` (v0.2.3) datasets collection
 * `climate4R.value` (v0.0.2) VALUE indices

To install them you may use the devtools package (e.g., ``devtools::install_github("SantanderMetGroup/downscaleR.keras@v1.0.0")`` to install `downscaleR.keras`). 

It is highly recommended that you use the *conda* package and environment manager to install the entire C4R framework (C4R version 1.5.0), executing the following commands in your command shell terminal: 
```shell
$ conda create --name climate4R
$ conda activate climate4R
$ conda install -c conda-forge -c r -c defaults -c santandermetgroup climate4r
```

*** 
**Note**: that even with C4R v1.5.0 installed via conda, you still need to upgrade libraries `climate4R.UDG` and `VALUE` with the `devtools` package by typing: 
```R
> devtools::install_github(c("SantanderMetGroup/climate4R.UDG@v0.2.2","SantanderMetGroup/VALUE@v2.2.2"))
```
***

We load the libraries with the specific versions needed to replicate the results.

In [ ]:
options(java.parameters = "-Xmx8g")  # expanding Java memory

# C4R libraries
library(loadeR)
library(loadeR.2nc)
library(transformeR)
library(downscaleR)
library(downscaleR.keras) 
library(visualizeR)
library(climate4R.value)

# Other useful libraries
library(magrittr)  # to pipe commands using '%>%' or '%<>%'

# For visualization purposes
library(RColorBrewer)
library(gridExtra)
library(ggplot2)

Intermediate data, predictions and models, generated along this notebook, are saved in a tree of directories created on the notebook's working directory. Please use the `dir.create` function to create two new directories (`data` and `models`) in your working directory. 
Within each of these directories, create subsequently two more subdirectories, named `tas` (near-surface temperature) and `pr` (precipitation) representing the respective predictands.

In [ ]:
dir.create("./data/pr", recursive = TRUE, showWarnings = FALSE)
dir.create("./data/tas", showWarnings = FALSE)
dir.create("./models/pr", recursive = TRUE, showWarnings = FALSE)
dir.create("./models/tas", showWarnings = FALSE)

Finally, create also a *figures* directory in the working directory.

In [ ]:
dir.create("./figures", showWarnings = FALSE)

Define here the **predictand** that will be used in the rest of the notebook. Predictor variables are defined in the next section, regardless of the predictand selected here.

In [ ]:
# --- define the predictand as 'pr' or 'tas' ---
predictand <- 'pr' 
# ----------------------------------------------

## Load and regrid data

We are now ready to load into our `R` environment all the data we are going to work with. The table below lists the required datasets, available in netCDF format,  which can be obtained in two ways:
* Access through the [Climate Data Service](http://meteo.unican.es/cds) developed by the [Santander Met Group](http://meteo.unican.es) (non registered users need to register first [here](http://meteo.unican.es/udg-tap/signup)). Use the `loginUDG` function to log into the service with your own credentials.
* Point to the url associated to each dataset to download the netCDF from the original source.

| Dataset/Label  | Data endpoint | License | Tracking id |
|---|---|---|---|
| ERA-Interim |  | | |
| E-OBS v20e | https://surfobs.climate.copernicus.eu/dataaccess/access_eobs.php#datafiles | | |
|CMIP5-subset_CanESM2_r1i1p1_historical |  | | |
|CMIP5-subset_CNRM-CM5_r1i1p1_historical |  | | |
|CMIP5-subset_MPI-ESM-MR_r1i1p1_historical |  | | |
|CMIP5-subset_MPI-ESM-LR_r1i1p1_historical |  | | |
|CMIP5-subset_NorESM1-M_r1i1p1_historical |  | | | 
|CMIP5-subset_GFDL-ESM2M_r1i1p1_historical |  | | |
|CMIP5-subset_EC-EARTH_r12i1p1_historical |  | | |
|CMIP5-subset_IPSL-CM5A-MR_r1i1p1_historical |  | | |
|CMIP5-subset_CanESM2_r1i1p1_rcp85 |  | | |
|CMIP5-subset_CNRM-CM5_r1i1p1_rcp85 |  | | |
|CMIP5-subset_MPI-ESM-MR_r1i1p1_rcp85 |  | | |
|CMIP5-subset_MPI-ESM-LR_r1i1p1_rcp85 |  | | |
|CMIP5-subset_NorESM1-M_r1i1p1_rcp85 |  | | |
|CMIP5-subset_GFDL-ESM2M_r1i1p1_rcp85 |  | | |
|CMIP5-subset_EC-EARTH_r12i1p1_rcp85 |  | | |
|CMIP5-subset_IPSL-CM5A-MR_r1i1p1_rcp85 |  | | |
|CORDEX-EUR-44_CCCma-CanESM2_historical_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_ETH-CLMcom-CCLM5-0-6_v1|  | | |
|CORDEX-EUR-44_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-CCLM4-8-17_v1|  | | |
|CORDEX-EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1|  | | |
|CORDEX-EUR-44_NCC-NorESM1-M_historical_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_ETH-CLMcom-CCLM5-0-6_v1|  | | |
|CORDEX-EUR-44_IPSL-IPSL-CM5A-MR_historical_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_IPSL-IPSL-CM5A-MR_historical_r1i1p1_IPSL-INERIS-WRF331F_v1|  | | |
|CORDEX-EUR-44_CCCma-CanESM2_rcp85_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_CNRM-CERFACS-CNRM-CM5_rcp85_r1i1p1_ETH-CLMcom-CCLM5-0-6_v1|  | | |
|CORDEX-EUR-44_CNRM-CERFACS-CNRM-CM5_rcp85_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1|  | | |
|CORDEX-EUR-44_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_MPI-CSC-REMO2009_v1|  | | |
|CORDEX-EUR-44_NCC-NorESM1-M_rcp85_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_NOAA-GFDL-GFDL-ESM2M_rcp85_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_ICHEC-EC-EARTH_rcp85_r12i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_ICHEC-EC-EARTH_rcp85_r12i1p1_ETH-CLMcom-CCLM5-0-6_v1|  | | |
|CORDEX-EUR-44_IPSL-IPSL-CM5A-MR_rcp85_r1i1p1_SMHI-RCA4_v1|  | | |
|CORDEX-EUR-44_IPSL-IPSL-CM5A-MR_rcp85_r1i1p1_IPSL-INERIS-WRF331F_v1|  | | |

Once these data are downloaded, the following block of code allows for loading the ERA-Interim predictor variables, which are needed to train our neural networks, for the period 1979-2005 by using the `loadGridData` function. Subsequently, the `makeMultiGrid` creates a unique C4R object containing all this information. **Note that we exemplify the data load from UDG, but the same commands can load from local files by replacing the `dataset` argument by the path to local data**.  

Moreover, since these loading steps can be quite time-consuming, we save the predictors `x` and predictand `y` data into R objects, which are also provided for reproducibility of the rest of the code.

In [ ]:
do.call(loginUDG, as.list(readLines("udg_login.txt")))
# or just:
#loginUDG("user","password")

In [ ]:
UDG.datasets("ECMWF_ERA-Interim-ESD", full.info = TRUE)

In [ ]:
# Predictor variables considered (see -*-)
vars  <- c(
  "psl",                         # Pressure at surface level 
  "z@500","z@700","z@850",       # Geopotential height at isobaric levels
  "hus@500","hus@700","hus@850", # Specific humidity at isobaric levels
  "ta@500","ta@700","ta@850",    # Air temperature at isobaric levels
  "ua@500","ua@700","ua@850",    # East-ward wind at isobaric levels 
  "va@500","va@700","va@850"     # North-ward wind at isobaric levels 
)

We loop over the variables and then use  makeMultiGrid,
to bind the variables in a single C4R object.
`ECMWF_ERA-Interim-ESD` is the path to the netCDFs 
of ERA-Interim-ESD downloaded from the endpoint 
indicated in the Table above.

In [ ]:
output.file = "./data/x.rds.xz"
if (! file.exists(output.file)){
  x <- lapply(vars, function(var) {
    loadGridData(
      dataset = "ECMWF_ERA-Interim-ESD",
      var     = var,
      lonLim  = c(-8,34), latLim  = c(34,76),  # domain of the predictors
      years   = 1979:2005
    )}
  ) %>% makeMultiGrid()
  saveRDS(x, output.file, compress="xz")
}

As predictands we use temperature and precipitation from E-OBS, which can be obtained as netCDF files [here](https://www.ecad.eu/download/ensembles/download.php).
Once downloaded, these data can be imported in `R` with the `loadGridData` function.
Subsequently, using `transformeR::interpGrid`, we upscale these E-OBS fields from their native 0.25º to the 0.5º regular grid our projections are delivered.
We illustrate next how to load either precipitation or temperature with `loadGridData`.

Retrieve E-OBS data if not available locally:

In [ ]:
eobs.varname <- switch(predictand, pr = "rr", tas = "tg")
eobs.data <- sprintf("./data/%s/%s_ens_mean_0.25deg_reg_v20.0e.nc", predictand, eobs.varname)
if (! file.exists(eobs.data)){
  download.file(
    sprintf("https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.25deg_reg_ensemble/%s_ens_mean_0.25deg_reg_v20.0e.nc", eobs.varname),
    eobs.data
  )
}

Upscale E-OBS:

In [ ]:
# boundaries of our projections domain and target resolution
grid05 = list("x" = c(-9.75,30.25),"y" = c(34.25,74.25))
attr(grid05,"resX") <- attr(grid05,"resY") <- 0.5 

# Load the predictand (y) and save to file
output.file = sprintf("./data/%s/y.rds.xz", predictand)
if (! file.exists(output.file)){
  y <- loadGridData(
    dataset = eobs.data,
    var = eobs.varname,
    lonLim = c(-10,30),
    latLim = c(34,75),
    years = 1979:2005
  ) %>% interpGrid(new.coordinates = grid05, method = "bilinear")
  saveRDS(y, output.file, compress="xz")
}

### Convolutional Neural Networks (CNNs)

To build *DeepESD* we rely on the convolutional neural networks (CNN) presented in [Baño-Medina et al. 2020](https://gmd.copernicus.org/articles/13/2109/2020/); in particular, on the CNN1 and CNN10 models, which were found to provide robust results for precipitation and temperature, respectively, both in ''perfect-prognosis'' conditions but also in the GCM space. The cell below shows how to build these CNN models based on `Keras`, and save them in a custom function called `modelCNN`. Note that precipitation and temperature CNN models are different. 
We refer the reader to [Baño-Medina et al. 2020](https://gmd.copernicus.org/articles/13/2109/2020/) for further details about the exact configuration of the CNNs used herein.

In [ ]:
modelCNN <- function(inp) {
  padding = switch(predictand, pr = "same", tas = "valid")
  filters.l1 = 50
  filters.l2 = 25
  filters.l3 = switch(predictand, pr = 1, tas = 10)
  # Input layer
  inputs <- layer_input(shape = dim(inp$x.global)[2:4])
  # Hidden layers
  l1 = layer_conv_2d(inputs, filters = filters.l1, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l2 = layer_conv_2d(    l1, filters = filters.l2, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l3 = layer_conv_2d(    l2, filters = filters.l3, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l4 = layer_flatten(    l3)
  # Output layer (depends on predictand)
  if (predictand == "pr") {
    l51 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'sigmoid') 
    l52 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear' ) 
    l53 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear' ) 
    outputs <- layer_concatenate(list(l51,l52,l53))
  } else if (predictand == "tas") {
    l51 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear') 
    l52 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear') 
    outputs <- layer_concatenate(list(l51, l52)) 
  }
  model <- keras_model(inputs = inputs, outputs = outputs) 
}

## DeepESD

In this section we 1) load the predictor variables of interest from the 8 GCM considered from the Santander CDS, 2) harmonize and standardize these predictor fields, 3) save these processed fields in `rda` objects to avoid repeating these steps in the future, 4) build the CNN models based on ERA-Interim predictors and E-OBS predictands and 5) apply these models to the GCM predictor variables to obtain the final high-resolution (downscaled at 0.5º) projections up to 2100.

### Preparing the predictor datasets

The following labels identify the 8 GCMs considered in this work in the Santander CDS, for the historical and RCP.8.5 scenario, respectively.
These labels are used to build the dataset names when calling the `loadGridData` function for data loading.

In [ ]:
## Use UDG.datasets()$CMIP5_subset to obtain the labels of the desired GCMs
model.names <- c(
  "CanESM2_r1i1p1",
  "CNRM-CM5_r1i1p1",
  "MPI-ESM-MR_r1i1p1",
  "MPI-ESM-LR_r1i1p1",
  "NorESM1-M_r1i1p1",
  "GFDL-ESM2M_r1i1p1",
  "EC-EARTH_r12i1p1",
  "IPSL-CM5A-MR_r1i1p1"
)

The following loop allows us to load the predictors from the above GCMs over our target domain for the reference (1975-2005) and future (early-future: 2006-2040, mid-future: 2041-2070, far-future: 2071-2100) periods of interest. Note that the historical experiment is used for the reference period and the RCP8.5 scenario for the future periods. Note also that, within the loop, all GCMs are interpolated to the spatial resolution of the ERA-Interim predictors which were used to fit the CNNs. Once loaded, the GCM predictors are saved as `.rda` files.

In [ ]:
fill850na <- function(x){
  # Since the IPSL contains NA values in the 850hPa level at certain gridpoints,
  # we replace these NA values with the numeric values of their closest neighbours.
  ind850 <- grepl("@850",x$Variable$varName,fixed = TRUE) %>% which()
  indGP <- apply(x$Data[ind850[1],1,,,], MARGIN = c(2,3), anyNA) %>% which(arr.ind = TRUE)
  for (i in 1:nrow(indGP)) {
    indTime <- is.na(x$Data[ind850[1],1,,indGP[i,1],indGP[i,2]]) %>% which()
    x$Data[ind850,1,indTime,indGP[i,1],indGP[i,2]] <- x$Data[ind850,1,indTime,indGP[i,1],indGP[i,2]-1]
  }
  return(x)
}

periods <- list(
  hist = 1975:2005, # must appear as first entry (used as reference)
  near = 2006:2040,
  midf = 2041:2070,
  farf = 2071:2100
)

In [ ]:
for (period in names(periods)){
  for (model.name in model.names) {
    output.file <- sprintf("./data/x_%s_%s.rds.xz",
      period,
      gsub("_.*","",model.name)
    )
    if (! file.exists(output.file)){
      if (period == "hist"){
        dataset.name <- sprintf("CMIP5-subset_%s_historical", model.name)
      } else {
        dataset.name <- sprintf("CMIP5-subset_%s_rcp85", model.name)
      }
      x.gcm <- lapply(vars, function(var) {
        loadGridData(
          dataset = dataset.name,
          var = var,
          lonLim = c(-8,34), latLim = c(34,76),
          years = periods[[period]]
        ) %>% interpGrid(new.coordinates = getGrid(x))
      }) %>% makeMultiGrid()
  
      if (substr(model.name,1,4) == "IPSL") {
        x.gcm <- fill850na(x.gcm)
      }
  
      saveRDS(x.gcm, file = output.file, compress = "xz")
    }
  }  
}  

The following loop allows us to harmonize and standardize the GCM predictors loaded in the previous step to assure they reasonable resemble the ERA-Interim variables used to train the CNN models (note this is one of the key assumptions that are done in ''perfect-prognosis'' downscaling). For this harmonization+standardization step, which is different depending on the particular scenario of interest (the reader is referred again to [Baño-Medina et al. 2020](https://gmd.copernicus.org/articles/13/2109/2020/) for details about this process), the `scaleGrid` function from `transformeR` is used. The so-processed GCM predictors, wich will be used as inputs to the CNN models, are saved as `rda` files. 

In [ ]:
ref.period <- 1979:2005

harmonize <- function(grid, base, ref){
  scaleGrid(grid, 
    base = base,
    ref = ref,
    type = "standardize",
    spatial.frame = "gridbox",
    time.frame = "monthly"
  ) 
}

standardize <- function(grid, base){
  scaleGrid(grid, 
    base = base,
    ref = ref,
    type = "standardize"
  ) 
}

In [ ]:
for (model.name in model.names) {
  model.basename <- gsub("_.*","",model.name)
  for (period in names(periods)) {
    raw.x <- loadRDS(sprintf("./data/x_%s_%s.rds.xz", period, model.basename))
    if (period == "hist") {
      harmonize.base <- subsetGrid(raw.x, years = ref.period)
      x.harm <- harmonize(raw.x, base = harmonize.base, ref = x)
      standardize.base <- subsetGrid(x.harm, years = ref.period)
    } else {
      x.harm <- harmonize(raw.x, base = harmonize.base, ref = x)
    }
    xn <- standardize(x.harm, base = standardize.base)
    # Save the standardized predictor fields as `rda` objects  
    saveRDS(xn,
      file = sprintf("./data/xn_%s_%s.rds.xz", period, gsub("_.*","",model.name)),
      compress="xz"
    )
    rm(xn)
  }
}

### Convolutional Neural Networks (CNNs)

To build *DeepESD* we rely on the convolutional neural networks (CNN) presented in [Baño-Medina et al. 2020](https://gmd.copernicus.org/articles/13/2109/2020/); in particular, on the CNN1 and CNN10 models, which were found to provide robust results for precipitation and temperature, respectively, both in ''perfect-prognosis'' conditions but also in the GCM space. The cell below shows how to build these CNN models based on `Keras`, and save them in a custom function called `modelCNN`. Note that precipitation and temperature CNN models are different. 
We refer the reader to [Baño-Medina et al. 2020](https://gmd.copernicus.org/articles/13/2109/2020/) for further details about the exact configuration of the CNNs used herein.

In [ ]:
modelCNN <- function(inp) {
  padding = switch(predictand, pr = "same", tas = "valid")
  filters.l1 = 50
  filters.l2 = 25
  filters.l3 = switch(predictand, pr = 1, tas = 10)
  # Input layer
  inputs <- layer_input(shape = dim(inp$x.global)[2:4])
  # Hidden layers
  l1 = layer_conv_2d(inputs, filters = filters.l1, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l2 = layer_conv_2d(    l1, filters = filters.l2, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l3 = layer_conv_2d(    l2, filters = filters.l3, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l4 = layer_flatten(    l3)
  # Output layer (depends on predictand)
  if (predictand == "pr") {
    l51 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'sigmoid') 
    l52 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear' ) 
    l53 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear' ) 
    outputs <- layer_concatenate(list(l51,l52,l53))
  } else if (predictand == "tas") {
    l51 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear') 
    l52 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear') 
    outputs <- layer_concatenate(list(l51, l52)) 
  }
  model <- keras_model(inputs = inputs, outputs = outputs) 
}

### Downscaling

This section shows how to fit the CNN model which links the large-scale predictors from ERA-Interim with the high-resolution E-OBS precipitation or temperature. The steps would be:
- Prepare the predictor and predictand tensors with the `prepareData.keras` function from `downscaleR.keras`.
- Standardize the ERA-Interim predictors using `trasformeR::scaleGrid`. 
- For precipitation, for a better fit of the Gamma distribution, 0.99 is substracted from observed precipitation and negative values are ignored (note that this step implies that rainy days are defined as those receiving 1 or more mm of precipitation). To do this, the `gridArithmetics` and `binaryGrid` functions from `transformeR` are used.
- Train the CNN model encapsulaled in the `modelCNN` function (previously defined) with the `downscaleTrain.keras` function from `downscaleR.keras`. To optimize the negative log-likelihood of the Bernoulli-Gamma distribution, we employ the custom loss function `bernouilliGammaLoss` from `downscaleR.keras`. The network is fitted using the Adam optimizer and a learning rate of 1e-4. Early-stopping with a patience of 30 epochs is applied and the best model (epoch) is saved in the working directory as a `.h5` file.

Load the reanalysis predictors and observed predictand saved previously:

In [ ]:
x <- readRDS("./data/x.rds.xz")
y <- readRDS(sprintf("./data/%s/y.rds.xz", predictand))

Standardize them depending on the target variable:

In [ ]:
x <- scaleGrid(x,type = "standardize")
if (predictand == "pr"){
  y <- binaryGrid(
    gridArithmetics(y, 0.99, operator = "-"),
    condition = "GE",
    threshold = 0,
    partial = TRUE
  )
}

Prepare predictor and predictand data for downscaling with `downscaleR.keras`

In [ ]:
xy.train <- prepareData.keras(
  x = x,
  y = y,
  first.connection = "conv",
  last.connection = "dense",
  channels = "last"
)

And train the model:

<div class="alert alert-block alert-warning">
<b>Warning:</b>
Running the next cell takes about 1 hour
</div>

In [ ]:
cnnmodel.name <- switch(predictand,
  pr="CNN1",
  tas="CNN10"
)
cnnloss <- switch(predictand,
  pr = bernouilliGammaLoss(last.connection = "dense"),
  tas = gaussianLoss(last.connection = "dense")
)

output.file <- sprintf('./models/%s/%s.h5', predictand, cnnmodel.name)
if (! file.exists(output.file)){

  # Training the CNN model
  downscaleTrain.keras(
    obj = xy.train, 
    model = modelCNN(xy.train),
    clear.session = TRUE,
    compile.args = 
      list(
        "loss" = cnnloss,
        "optimizer" = optimizer_adam(lr = 0.0001)
      ),
    fit.args =
      list(
        "batch_size" = 100,
        "epochs" = 10000,
        "validation_split" = 0.1,
        "verbose" = 1,
        "callbacks" = list(
           callback_early_stopping(patience = 30),
           callback_model_checkpoint(
             filepath = output.file,
             monitor = 'val_loss',
             save_best_only = TRUE
           )
        )
     )
  )

}

Once the model is trained, we use it to predict in both the train (training period using ERA-Interim variables) and the GCM spaces. As per the former, for precipitation, we are interested in the estimation of the parameter `p` (probability of rain), since it is needed later to adjust the frequency of rain in the high-resolution projections obtained from the GCM (see the manuscript for details).

To compute `p` in the train period the following is done:
- Prepare the predictors which will serve as inputs for the CNN model with the `prepareNewData.keras` function. Subsequently, use them to predict in the train set with the `downscalePredict.keras` function. The `model` argument indicates the path where the CNN model was previously stored, and `C4Rtemplate` is a C4R object used as template for the predictions which provides the proper metadata. Since `downscalePredict.keras` outputs 3 parameters (the probability of rain, `p`, and the logarithmic of the shape and scale parameters of the Gamma distribution, `log_alpha` and `log_beta`), the `subsetGrid` is applied in order to keep only `p`.

In [ ]:
cnnloss.name <- switch(predictand,
  pr = "bernouilliGammaLoss",
  tas = "gaussianLoss"
)

cnn.predict <- function(newdata){
  downscalePredict.keras(
    newdata = newdata,
    C4R.template = y,
    clear.session = TRUE,
    loss = cnnloss.name,
    model = list(
      "filepath" = sprintf('./models/%s/%s.h5', predictand, cnnmodel.name),
      "custom_objects" = c(
          "custom_loss" = cnnloss
      )
    )
  )  
}

In [ ]:
if (predictand == "pr"){
  xy.test <- prepareNewData.keras(x, xy.train)
  pred_ocu_train <- cnn.predict(xy.test) %>% subsetGrid(var = "p")
}

At this point, the trained CNN model is used to generate the high-resolution projections building on the 8 GCMs considered in this work. To do so, we perform a loop over the GCMs in which the corresponding predictors (which had been previously saved) are loaded and conveniently transformed using the `prepareNewData.keras` function. Finally, the `log_alpha`, `log_beta` and `p` parameters, which are obtained with the `downscalePredict.keras` function are saved in the `pred` object.
- On the one hand, `log_alpha` and `log_beta` are used to obtain the rainfall amount with the `computeRainfall` function from `downscaleR.keras`. The argument `simulate` allows us for specifying if either a stochastic or a deterministic outcome is wanted. The argument `bias` is used to re-center the Gamma distribution to 1mm/day. 
- On the other hand, we use the `p` parameter to derive the binary event occurrence/non occurrence through the `bynaryGrid` function.
- Finally, both series (binary and continuous) are multiplied to produce the complete precipitation time series.

In the following block of code we define commodity functions to compute the (deterministic) frequency and (stochastic) amount of rain, or the deterministic temperature value (as the mean of the predicted distribution). Then we iterate to predict (downscale) for all models, which are saved in `.nc` format with the `grid2nc` function.

In [ ]:
predict.pr <- function(pred.obj){
  # Deterministic frequency and stochastic amount of rain
  computeRainfall(
    log_alpha = subsetGrid(pred.obj, var = "log_alpha"),
    log_beta  = subsetGrid(pred.obj, var = "log_beta"),
    bias = 1,
    simulate = TRUE
  ) %>% gridArithmetics(
    binaryGrid(
      subsetGrid(pred.obj, var = "p"),
      ref.obs = binaryGrid(y, threshold = 1, condition = "GE"),
      ref.pred = pred_ocu_train
    )
  )  
}

predict.tas <- function(pred.obj){
  ## Deterministic version 
  rval <- subsetGrid(pred.obj, var = "mean")
  rval$Variable$varName <- "tas"
  return(rval)
}

<div class="alert alert-block alert-warning">
<b>Warning:</b>
Running the next cell takes hours
</div>

In [ ]:
for (model.name in model.names) {
  model.basename <- gsub("_.*","",model.name)    
  for (period in names(periods)) {
    xn <- loadRDS(sprintf("./data/xn_%s_%s.rds.xz", period, model.basename))
    xy.test <- prepareNewData.keras(xn, xy.train)
    pred <- cnn.predict(xy.test)
    pred.params <- switch(predictor,
      pr = predict.pr(pred),
      tas = predict.tas(pred)
    ) 
    grid2nc(pred.params, NetCDFOutFile = sprintf("./data/%s/y_CNN_%s_%s.nc4", period, model.basename))   
  }
}

## Dynamical climate models
To assess the credibility of DeepESD, it is compared against two different ensembles of dynamical models (see the *Technical Validation* section), the first/second of them formed by Global/Regional Climate Models (GCMs/RCMs). Since DeepESD covers only land, we start by creating a 0.5º land-sea mask which will be later applied to eliminate sea points from both GCMs and RCMs.

In [ ]:
output.file <- "./data/mask.nc4"
if (! file.exists(output.file)){

  y = readRDS("./data/pr/y.rds.xz")
  mask <- gridArithmetics(subsetGrid(y, year = 1990), 0) %>% gridArithmetics(1, operator = "+") %>% climatology()
  mask$Variable$varName <- "sftlf"
  grid2nc(mask, NetCDFOutFile = output.file)

} else{
    
  mask <- loadGridData(output.file, var = "sftlf")

}

### Ensemble of Global Climate Models (GCMs)

We perform a loop over the temporal periods of interest (1975-2005 for the historical scenario plus 2006-2040, 2041-2070 and 2071-2100 for RCP8.5) and save the GCM ensemble as netCDF files (`grid2nc` function) in a multi-member C4R object. All GCMs are interpolated to our target 0.5º resolution (E-OBS grid), using conservative remapping. To do this interpolation, we rely on the `cdo` library and use function `system` to invoke the OS command. Please note that you can install the `cdo` library with conda, by typing `conda install cdo` in a terminal. Finally, sea points are removed by applying the land-sea mask we have previously created.

In [ ]:
cdo.conservative.remap <- function(grid){
  grid2nc(grid, NetCDFOutFile = "./aux.nc4")
  system("cdo remapcon,data/mask.nc4 ./aux.nc4 ./aux2.nc4")
  rval <- loadGridData("./aux2.nc4", var = predictand)
  file.remove(c("./aux.nc4","./aux2.nc4"))
  return(rval)
}

mask.landsea <- function(grid){
  lapply(1:getShape(grid, "time"), function(t) {
    gridArithmetics(
      subsetDimension(grid, dimension = "time", indices = t),
      mask
    )
  }) %>% bindGrid(dimension = "time")
}

In [ ]:
for (period in names(periods)){
  for (model.name in model.names) {
    output.file <- sprintf("./data/%s/y_GCM_%s_%s.nc4",
      predictand,
      period,
      gsub("_.*","",model.name)
    )
    if (! file.exists(output.file)){
      if (period == "hist"){
        dataset.name <- sprintf("CMIP5-subset_%s_historical", model.name)
      } else {
        dataset.name <- sprintf("CMIP5-subset_%s_rcp85", model.name)
      }
      # Load the data and interpolate to the target resolution
      y.gcm <- cdo.conservative.remap(loadGridData(
          dataset = dataset.name,
          var = predictand,
          lonLim = c(-10,30), latLim = c(34,74),
          years = periods[[period]]
      ))
      y.gcm <- mask.landsea(y.gcm)
      grid2nc(y.gcm,NetCDFOutFile = output.file)
    }
  }  
}

### Ensemble of Regional Climate Models (RCMs) 
In this section we form an ensemble of EURO-CORDEX RCMs which can be easily loaded from the Santander CDS by using the appropiate labels (see the block below).

In [ ]:
# Labels for the historical scenario
rcm.labels.hist <- c(
  "CORDEX-EUR-44_CCCma-CanESM2_historical_r1i1p1_SMHI-RCA4_v1",
  "CORDEX-EUR-44_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_ETH-CLMcom-CCLM5-0-6_v1",
  "CORDEX-EUR-44_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_SMHI-RCA4_v1",
  "CORDEX-EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-CCLM4-8-17_v1",
  "CORDEX-EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1",
  "CORDEX-EUR-44_NCC-NorESM1-M_historical_r1i1p1_SMHI-RCA4_v1",
  "CORDEX-EUR-44_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_SMHI-RCA4_v1",
  "CORDEX-EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1",
  "CORDEX-EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_ETH-CLMcom-CCLM5-0-6_v1",
  "CORDEX-EUR-44_IPSL-IPSL-CM5A-MR_historical_r1i1p1_SMHI-RCA4_v1",
  "CORDEX-EUR-44_IPSL-IPSL-CM5A-MR_historical_r1i1p1_IPSL-INERIS-WRF331F_v1"
)

get.rcm.simulation.id <- function(x){
  gcm <- unlist(strsplit(x,"_"))[2]
  gcm <- unlist(strsplit(gcm,"-"))[-1]
  if (gcm[[1]] %in% c("M", "CERFACS", "GFDL")) # Drop modelling center remnants
    gcm <- gcm[-1]
  gcm <- paste(gcm, collapse="-")
  rcm <- unlist(strsplit(x,"_"))[5]
  rcm <- unlist(strsplit(rcm,"-"))[-1]
  if (rcm[[1]] %in% c("INERIS", "CLMcom", "CSC")) # Drop modelling center remnants
    rcm <- rcm[-1]
  rcm <- paste(rcm, collapse="-")
  sprintf("%s_%s", gcm, rcm)
}

# e.g.:
get.rcm.simulation.id(rcm.labels.hist[2])

We perfrom a loop over the temporal periods of interest (1975-2005 for the historical scenario plus 2006-2040, 2041-2070 and 2071-2100 for RCP8.5) and save the RCM ensemble as netCDF files (`grid2nc` function) in a multi-member C4R object. All RCMs are interpolated to our target 0.5º resolution (E-OBS grid) and sea points are removed by applying the land-sea mask we have previously created.

In [ ]:
for (period in names(periods)){
  for (dataset.name in rcm.labels.hist) {
    output.file <- sprintf("./data/%s/y_RCM_%s_%s.nc4",
      predictand,
      period,
      get.rcm.simulation.id(dataset.name)
    )
    if (! file.exists(output.file)){
      if (! period == "hist"){
        dataset.name <- gsub("historical","rcp85", dataset.name)
      }
      # Load the data and interpolate to the target resolution
      y.rcm <- loadGridData(
          dataset = dataset.name,
          var = predictand,
          lonLim = c(-10,30), latLim = c(34,74),
          years = periods[[period]]
      ) %>% interpGrid(getGrid(mask))
      y.rcm <- mask.landsea(y.rcm)  
      grid2nc(y.rcm,NetCDFOutFile = output.file)
    }
  }  
}

## Results

This section provides the code needed to reproduce the figures presented in the manuscript. Note we mostly rely on the `visualizeR` package for plotting since it supports both spatial maps and temporal series.

### Ensemble mean and bias with respect to E-OBS

Figure 1 in the manuscript shows the climatology of the different ensembles built (GCM, RCM and DeepESD), along with the corresponding mean error (bias) with respect to the observed pattern in the historical period. We start thus by computing the climatology of the different contributing members forming each ensemble and saving them as netCDF files in the working directory.

In [ ]:
# because a member of the RCM ensemble misses a value on 01-Jan-2006, so to preserve
# temporal consistency in the ensemble we add this date to the ensemble mean metadata
near.dates <- list(
  start = "2006-01-01 12:00:00 GMT",
    end = "2041-01-01 12:00:00 GMT"
)
model.types <- c("GCM","RCM","CNN")
gcm.basenames <- gsub("_.*","",model.names)
rcm.basenames <- unlist(lapply(rcm.labels.hist, get.rcm.simulation.id))

In [ ]:
for (model.type in model.types){
  basenames = ifelse(model.type == "RCM", rcm.basenames, gcm.basenames)
  for (period in names(periods)){
    output.file <- sprintf("./data/%s/y_%s_%s_ensemble.nc4", predictand, model.type, period)
    if (! file.exists(output.file)){

      ens.mean <- lapply(basenames, FUN = function(basename) {
        path <- sprintf("./data/%s/y_%s_%s_%s.nc4", predictand, model.type, period, basename)
        grid <- loadGridData(dataset = path, var = predictand)
        grid <- valueIndex(grid, index.code = "Mean")$Index  # compute mean of each member 
        if (period == "near") grid$Dates <- near.dates 
        return(grid)  
      }) %>% bindGrid(dimension = "member")  # bind member means in a single C4R object along the `member` dimension    
      ens.mean$InitializationDates <- NULL
      grid2nc(ens.mean, NetCDFOutFile = output.file)
        
    }
  }
}

Next, the mean climatology for each ensemble is obtained with the `aggregateGrid` function (note the aggregation is done along the member dimension) from `transformeR`. Afterwards, we can already use `spatialPlot` to plot the corresponding spatial pattern. The resulting figure is saved in `pdf` format in the path indicated in the `pdfOutput` object.

In [ ]:
if (predictand == "pr") {
  palette <- "BuPu"
  at <- seq(0,8,0.5)
  units <- "mm/day"
} else if (predictand == "tas") {
  palette <- "OrRd"
  at <- seq(-5, 20,2.5)
  units <- "ºC"
}

cb <- c("#FFFFFF",brewer.pal(n = 9, palette))
cb <- cb %>% colorRampPalette()
pdfOutput <- sprintf("./figures/ensembleMean_%s.pdf" , predictand)

In [ ]:
nn <- c("GCM","RCM","CNN")
figs <- lapply(1:length(nn), FUN = function(z) {
  
  # We store in `grid` object the ensemble of climatologies  
  grid <- loadGridData(paste0("./data/", predictand, "/", nn[z], "_h_ensemble.nc4"), var = "Mean")
  
  # Compute the ensemble mean  
  gridMean <-  aggregateGrid(grid,aggr.mem = list(FUN = "mean", na.rm = TRUE)) 
  
  # We depict the ensemble mean with spatialPlot function  
  spatialPlot(
    gridMean,
    backdrop.theme = "coastline",
    main = paste0("Ensemble Mean (",units,") - ",nn[z]),
    ylab = "1975-2005",
    col.regions = cb,
    at = at,
    set.min = at[1], 
    set.max = at[length(at)])
})

pdf(pdfOutput, width = 15, height = 10)   
grid.arrange(grobs = figs, ncol = 3)                     
dev.off()

Now we plot the bias with respecto to the observed (i.e. E-OBS) climatology. Again, we rely on `spatialPlot` to depict the spatial fields, and `aggregateGrid` and `gridArithmetics`, to compute the ensemble mean and its bias, respectively. The resulting figures are saved in `pdf` format in the path indicated by `pdfOutput`.

In [ ]:
## Please select one:   -----------------------------------------------------------------------
# Precipitation
cb <- brewer.pal(n = 11, "BrBG")
cb[6] <- "#FFFFFF"; cb <- cb %>% colorRampPalette()
at <- c(seq(-2, -0.5,0.5),-0.25,0.25,seq(0.5, 2,0.5))    
units <- "mm/day"
pdfOutput <- "./figures/bias_pr.pdf" 
var <- "precip"
load("./data/pr/y.rda")

In [ ]:
# Temperature
cb <- rev(brewer.pal(n = 11, "RdBu"))
cb[6] <- "#FFFFFF"; cb <- cb %>% colorRampPalette()
at <- c(seq(-2, -0.5,0.5),-0.25,0.25, seq(0.5,2,0.5)) 
units <- "ºC"
pdfOutput <- "./figures/bias_tas.pdf"
var <- "temperature"
load("./data/tas/y.rda")
## --------------------------------------------------------------------------------------------

In [ ]:
nn <- c("GCM","RCM","CNN")
figs <- lapply(1:length(nn), FUN = function(z) {
  # Compute the ensemble mean  
  grid <- loadGridData(paste0("./data/",var,"/",nn[z],"_h_ensemble.nc4"), var = "Mean") %>% aggregateGrid(aggr.mem = list(FUN = "mean", na.rm = TRUE))
  # Compute the bias with respect to the observed temporal climatology for the same period
  grid  %<>% gridArithmetics(valueIndex(y, index.code = "Mean")$Index,operator = "-")
  # Depict the bias of the ensemble mean
  spatialPlot(grid,
              backdrop.theme = "coastline",
              main = paste0("Bias Ensemble Mean (",units,") - ",nn[z]),
              ylab = "1975-2005",
              col.regions = cb,
              at = at,
              set.min = at[1], set.max = at[length(at)])  
}) 
pdf(pdfOutput, width = 15, height = 10)   
grid.arrange(grobs = figs, ncol = 3)                     
dev.off()

### Climate change signals

To produce Figure 2 in the manuscript we perform a loop, for each ensemble (GCM, RCM and DeepESD), over the different RCP8.5 periods of interest and sequentially compute the difference between the future climatology and the historical one. These climate change signals are then averaged along the member dimension using the `aggregateGrid` function. The resulting figures are saved in `pdf` format in the path indicated by `pdfOutput`.

In [ ]:
## Please select one:   -----------------------------------------------------------------------
# Precipitation
cb <- brewer.pal(n = 11, "BrBG")
cb[6] <- "#FFFFFF"; cb <- cb %>% colorRampPalette()
at <- c(seq(-1, -0.25,0.25),-0.125,0.125,seq(0.25, 1,0.25)) 
pdfOutput <- "./figures/deltas_pr.pdf" 
var <- "precip"

In [ ]:
# Temperature
cb <- c("#FFFFFF",brewer.pal(n = 11, "OrRd"))
at <- c(seq(0,4,0.5),5,6)
pdfOutput <- "./figures/deltas_tas.pdf"
var <- "temperature"
## --------------------------------------------------------------------------------------------

In [ ]:
nn <- c("GCM","RCM","CNN")
figs <- lapply(c("ef","mf","ff"), FUN = function(z) {  
    lapply(1:length(nn), FUN = function(zz) {
    gridh <- loadGridData(paste0("./data/",var,"/",nn[zz],"_h_ensemble.nc4"), var = "Mean") 
    gridf <- loadGridData(paste0("./data/",var,"/",nn[zz],"_",z,"_ensemble.nc4"), var = "Mean") 
    grid <- gridArithmetics(gridf,gridh,operator = "-")
    gridMean <-  aggregateGrid(grid,aggr.mem = list(FUN = "mean", na.rm = TRUE)) 

    if (z == "ef") period <- c("2006-2040")
    if (z == "mf") period <- c("2041-2070")
    if (z == "ff") period <- c("2071-2100")
    spatialPlot(gridMean,
                backdrop.theme = "coastline",
                main = paste("CC. signal wrt 1975-2005"),
                ylab = period,
                col.regions = cb,
                at = at,
                set.min = at[1], set.max = at[length(at)])  
  }) 
}) %>% unlist(recursive = FALSE)   
pdf(pdfOutput, width = 15, height = 10)   
grid.arrange(grobs = figs, ncol = 3)                     
dev.off()

### Time-series

The following block of code allows for plotting the time-series of the climate change signals. For precipitation (temperature), we perform a loop over the validation metrics of interest: R01, SDII (Mean). For details about these metrics please see the manuscript or type `show.indices()` in a new cell. At each iteration of the loop we define a `doCall.args` list which contains the `aggr.y` arguments needed for the `aggregateGrid` function (note the validation is done at an annual basis), which is finally passed t  `do.call`. At the end of the loop, the resulting figures are saved in `pdf` format.

In [ ]:
## Please select one:   -----------------------------------------------------------------------
# Precipitation
indices <- c("R01","SDII")
variable  <- "precip"
var <- "pr"
load("./data/pr/y.rda")

In [ ]:
# Temperature
indices <- c("Mean")
variable <- "temperature"
var <- "tas"
load("./data/tas/y.rda")
## --------------------------------------------------------------------------------------------

In [ ]:
dName <- c("CanESM2","CNRM-CM5","MPI-ESM-MR","MPI-ESM-LR","NorESM1","GFDL","EC-Earth","IPSL")

In [ ]:
figs <- lapply(indices, FUN = function(zz) {
  doCall.args <- list() 
  doCall.args[["aggr.y"]] <- list()
  
  # The R01 do.call arguments  
  if (zz == "R01")  {
    doCall.args[["aggr.y"]][["FUN"]] <- "index.freq"
    doCall.args[["aggr.y"]][["freq.type"]] <- "rel"
    doCall.args[["aggr.y"]][["condition"]] <- "GE"
    doCall.args[["aggr.y"]][["threshold"]] <- 1
    ylim <- c(0.24,0.54)  
  }
  # The SDII do.call arguments  
  if (zz == "SDII"){
    doCall.args[["aggr.y"]][["FUN"]] <- "index.meanGE"
    doCall.args[["aggr.y"]][["threshold"]] <- 1
    ylim <- c(2,9)  
  } 
  # The Mean do.call arguments    
  if (zz == "Mean"){
    doCall.args[["aggr.y"]][["FUN"]]   <- "mean"
    doCall.args[["aggr.y"]][["na.rm"]] <- TRUE
    ylim <- c(0,18)  
  }    
    
  # We compute the index for the GCM ensemble. To do this, we loop over the temporal periods and then bind the serie along the time dimension with bindGrid function    
  pred1 <- lapply(c("h","ef","mf","ff"), FUN = function(z) { 
    lapply(1:length(dName), FUN = function(zzz) {            
      doCall.args[["grid"]] <- loadGridData(dataset = paste0("./data/",predictand,"/y_",z,"_",dName[zzz],".nc4"),var = var)
      do.call("aggregateGrid",doCall.args)
    }) %>% bindGrid(dimension = "member")
  }) %>% bindGrid(dimension = "time") 
  
  # We compute the index for the RCM ensemble. To do this, we loop over the temporal periods and then bind the serie along the time dimension with bindGrid function
  pred3 <- lapply(c("h","ef","mf","ff"), FUN = function(z) { 
    lapply(1:11, FUN = function(zzz) {            
      doCall.args[["grid"]] <- loadGridData(dataset = paste0("./data/",predictand,"/yRCM_",z,"_member",zzz,".nc4"),var = var)
      do.call("aggregateGrid",doCall.args)
    }) %>% bindGrid(dimension = "member")
  }) %>% bindGrid(dimension = "time")
    
    # We compute the index for the DeepESD ensemble. To do this, we loop over the temporal periods and then bind the serie along the time dimension with bindGrid function    
  pred2 <- lapply(c("h","ef","mf","ff"), FUN = function(z) { 
    lapply(1:length(dName), FUN = function(zzz) {            
      doCall.args[["grid"]] <- loadGridData(dataset = paste0("./data/",predictand,"/CNN_",z,"_",dName[zzz],".nc4"),var = var)
      do.call("aggregateGrid",doCall.args)
    }) %>% bindGrid(dimension = "member")
  }) %>% bindGrid(dimension = "time")
      
  # We compute the index for the observed temporal serie 
  doCall.args[["grid"]] <- y
  y <- do.call("aggregateGrid",doCall.args)
  
  # We call temporalPlot to plot the times-series 
  temporalPlot("OBS" = y,"GCM" = pred1,"RCM" = pred3,"CNN" = pred2, cols = c("black","red","blue","green"),xyplot.custom = list(ylim = ylim))       
})


In [ ]:
# Saving the resulting figures in .pdf format
pdf(paste0("./figures/serie_",var,".pdf"), width = 15, height = 4)
grid.arrange(grobs = figs, ncol = 3)  
dev.off()